In [1]:
import pandas as pd
import ast
import joblib
import math
import numpy as np
from shapely.geometry import Point, LineString
from datetime import datetime,timedelta
import folium

4) Algorithm for calculating the bus position (percentage of route completed), current delays, and estimated time to the next stop for a given trip_id. It also checks if the bus is currently at a stop.

Preparing Data for Algorithm Validation – we need to adjust the timestamp to account for local time and the winter time change. In the final version (in the application), data will be streamed to the algorithm via Kafka and processed in real-time.

We are preparing data on bus trips (GPS readings) for November 11, 2025, from 16:00 to 23:00

In [ ]:
bus_stops = pd.read_csv('results\\stops_modified.txt')
bus_stops['arrival_time'] = pd.to_datetime(bus_stops['arrival_time'])
bus_stops['departure_time'] = pd.to_datetime(bus_stops['departure_time'])
trips = pd.read_csv('gdansk_04_18_11_2024\\trips.txt')

gps_data2 = pd.read_csv("GTFS-RT data.csv", sep=",", header=None, names=['zoned_timestamp', 'city', 'trip_id', 'vehicle_id', 'vehicle_label', 'longitude', 'latitude', 'speed', 'vehicle_service', 'gps_quality', 'delay', 'bearing', 'current_stop_sequence']) #surowe dane z próbki
gps_data2['timestamp'] = gps_data2['zoned_timestamp']
gps_data2 = gps_data2.drop(columns='zoned_timestamp')
gps_data2['timestamp'] = pd.to_datetime(gps_data['timestamp']).dt.tz_localize(None)
gps_data2['timestamp'] = gps_data2['timestamp'] + pd.Timedelta(hours=1) #We add one due to the time change—overall, it should be +2
gps_data2 = gps_data2.drop_duplicates()

segments = pd.read_csv("results\\segments.txt")
segments['coordinates'] = segments['coordinates'].apply(ast.literal_eval)

kdtree_dict_loaded = joblib.load('results\\kdtree_with_shape_ids.joblib')

In [ ]:
#gps_data2 = gps_data2[gps_data2.trip_id == '283202411051446_62_283-01'] #if u want specific trip

simulating data streaming – each GPS reading is treated as an object containing the recorded timestamp, coordinates, and trip_id

In [19]:
class DataPoint:
    def __init__(self, timestamp, gps_coords, trip_id, bearing=None):
        """
        Initializes a DataPoint.
        :param timestamp: The timestamp of the data point.
        :param gps_coords: Tuple of (latitude, longitude).
        :param trip_id: ID of the trip.
        :param bearing: Precomputed bearing (optional).
        """
        self.timestamp = timestamp
        self.gps_coords= gps_coords
        self.trip_id = trip_id
        self.bearing = bearing

    def __repr__(self):
        """String representation of the DataPoint."""
        return f"DataPoint(timestamp={self.timestamp}, gps_coords={self.gps_coords}, trip_id={self.trip_id}, bearing={self.bearing})"

    def get_latitude(self):
        """Returns the latitude from gps_coords."""
        return self.gps_coords[0] if self.gps_coords else None

    def get_longitude(self):
        """Returns the longitude from gps_coords."""
        return self.gps_coords[1] if self.gps_coords else None

    def get_bearing(self):
        """Returns the precomputed bearing."""
        if self.bearing is not None:
            return self.bearing
        else:
            return "Bearing information not available"

    def get_trip_info(self):
        """Returns a summary of the trip information."""
        return f"Trip ID: {self.trip_id} at {self.timestamp} with coordinates {self.gps_coords}"


#point1 = DataPoint("2024-10-25 00:00:13", [54.356060, 18.645491], "401202410242331_21_401-02") #example of DataPoint object

In [20]:
def euclidian_distance_in_km(coords1, coords2): # due to the small area, we simplify the calculations and use Euclidean distance 
    delta_lat = coords1[0] - coords2[0]
    delta_lon = coords1[1] - coords2[1]  
    # conversion to kilometers
    delta_lat_km = delta_lat * 111.32  # 1 degree of latitude ≈ 111.32 km
    delta_lon_km = delta_lon * 69.5    # 1 degree of longitude ≈ 69.5 km near Warsaw
    
    return math.sqrt(delta_lat_km**2 + delta_lon_km**2)


def point_to_segment_distance(coords1, coords2, coords3): # calculating the Euclidian distance from a bus point to a line formed by consecutive points from shapes
    px, py = coords1
    x1, y1 = coords2
    x2, y2 = coords3
    px *= 111.32
    x1 *= 111.32
    x2 *= 111.32
    py *= 69.5
    y1 *= 69.5
    y2 *= 69.5
    point = Point(px, py)
    line = LineString([(x1, y1), (x2, y2)])

    return point.distance(line)

main algorithm:

In [21]:
def gps_processing_algorithm(data_point, bus_stops, kdtree_dict, segments, history):
    
    gps_coords = data_point.gps_coords #coordinates of gps reading   
    timestamp = pd.to_datetime(data_point.timestamp)
    bus_stops = bus_stops[bus_stops['trip_id'] == data_point.trip_id]
    coords2 = bus_stops.loc[:, ["stop_lat", "stop_lon"]].values.tolist() #stops coordinates
    shape_id = bus_stops.iloc[0]['shape_id']
    
    # if not shape_id.empty:        
    #     shape_id = shape_id.iloc[0]['shape_id']
    # else:
    #     return np.nan
    
    segments = segments[segments['shape_id'] == shape_id]
    tree, coords3, distance1 = kdtree_dict[shape_id]
    total_distance = distance1[-1]
    coords3 = [ast.literal_eval(x) if isinstance(x, str) else x for x in coords3]
    
    #creating a k-d tree from the coordinates of lines starting points
    query_point = np.array(gps_coords).reshape(1, -1)
    indices = tree.query(query_point, k=5)[1][0] #5 nearest neighbours for each gps reading - quick solution

    SEGMENT, ACCUMULATED_DISTANCE = np.array([]), np.array([])   
    keys = ['trip_id', 'timestamp', 'coords', 'segment', '%', 'bus_stop_number', 'expected_arrival_time', 'delay','accumulated_distance']
    output= {key: None for key in keys}    
    output['trip_id'] = data_point.trip_id
    output['timestamp'] = timestamp
    output['coords'] = gps_coords
    

    array = np.array([])    
    
    for j in range(len(indices)): #iterating over the closest neighbours of the i-th GPS reading  
        dist = point_to_segment_distance(gps_coords, coords3[indices[j]], coords3[indices[j]+1]) * 1000 #calculating the distance from the i-th GPS reading to the segment in pairs_df, based on the starting point that is the nearest neighbor of the GPS point
        array = np.append(array, dist)
    
    if(min(array) > 70): #the trip is not following the planned route (too far from the closest line)
        return np.nan
    

    line_numbers = indices[np.argsort(array)[:3]] #potential 3 lines where the GPS reading is located (specifically, the nearest points from the coor3 list)
    
    for number in line_numbers:
        lin = [coords3[number], coords3[number + 1]]
        for j in range(len(segments)): #segment detection
            coords_list = segments.coordinates.iloc[j]
            if (lin[0] in coords_list and lin[1] in coords_list):      
                SEGMENT = np.append(SEGMENT, segments.id.iloc[j])
                break
    
        dist2 = distance1[number] + euclidian_distance_in_km(gps_coords, coords3[number]) #distance accumulated by bus so far
        ACCUMULATED_DISTANCE = np.append(ACCUMULATED_DISTANCE, dist2)
 
    first_three_values = array[np.argsort(array)[:3]]  #3 smallest values
    #print([historia, ACCUMULATED_DISTANCE, dokladna_data_dt])

    if np.isnan(history).all(): #we dont have history of that bus yet - taking minimum from 2 smallest values
        ACCUMULATED_DISTANCE = ACCUMULATED_DISTANCE[:2] 
        SEGMENT, ACCUMULATED_DISTANCE = SEGMENT[[np.argmin(ACCUMULATED_DISTANCE)]], ACCUMULATED_DISTANCE[[np.argmin(ACCUMULATED_DISTANCE)]]
    
    elif first_three_values[0] == first_three_values[1]:  #the bus passes through a given line twice, and we need to determine at which point in the route we currently are - taking the gps the closest to history
      
        idx = np.argmin(np.abs(ACCUMULATED_DISTANCE - history[0]))
        SEGMENT, ACCUMULATED_DISTANCE = SEGMENT[[idx]], ACCUMULATED_DISTANCE[[idx]]
    
    else:
        for i in range(3):

            if (0.5 > (ACCUMULATED_DISTANCE[[i]] - history[0]) > -0.015): #If we move back less than 15 meters, we choose this index; otherwise, we continue iterating
                
                SEGMENT, ACCUMULATED_DISTANCE = SEGMENT[[i]], ACCUMULATED_DISTANCE[[i]]
                break
        else:
            
            SEGMENT, ACCUMULATED_DISTANCE = SEGMENT[[0]], ACCUMULATED_DISTANCE[[0]] #taking first closest result
   


    output['segment'], output['accumulated_distance'] = SEGMENT, ACCUMULATED_DISTANCE 
    route_percentage = ACCUMULATED_DISTANCE[0] / total_distance 
    
    delay, expected_arrival_time = np.nan, []
    for j in range(len(coords2)): #iterating over bus stops

        if euclidian_distance_in_km(gps_coords, coords2[j]) <  0.1: #we are in the surrounding area of a bus stop
            if j == 0:
                output['%'] = 0
            elif j == len(coords2) -1:
                output['%'] = 1
            else:
                output['%'] = min(1, route_percentage)

            output['bus_stop_number'] = f"STOP {j+1}"
            expected_arrival_time.append(0) # in case when we reached bus stop first expected time is equal to 0
            
            if j != len(coords2) - 1:    
                diff = bus_stops.arrival_time.iloc[j+1] - timestamp
                expected_arrival_time.append(diff.total_seconds()/60)    
            if expected_arrival_time[-1] < 0 : #explicit delay information
                delay = abs(expected_arrival_time[-1])
                
            output['expected_arrival_time'], output['delay'] = expected_arrival_time, delay     
            
            return output
        
        
        elif float(bus_stops.iloc[j]["stop_dist_traveled"]) <= ACCUMULATED_DISTANCE[0] <= float(bus_stops.iloc[min(j+1, len(bus_stops)-1)]["stop_dist_traveled"]): #we are somewhere between bus stops, outside their close range
            
            output['%'] = min(1, route_percentage)
                 
            diff = bus_stops.arrival_time.iloc[j] - timestamp
            expected_arrival_time.append(diff.total_seconds()/60)

            if expected_arrival_time[-1] < 0 : #explicit delay information
                    delay = abs(expected_arrival_time[-1])
         
            output['expected_arrival_time'], output['delay'] = expected_arrival_time, delay      

            return output

    return output

Tests – we process the time period from 2024-11-05 16:00 to 2024-11-05 23:00 "reading by reading" (we do not necessarily need to process the entire period, as it takes quite a long time). As a result, we obtain the `processed_gps` dataframe, which collects the history of processed points. 

In [ ]:
processed_gps = pd.DataFrame(columns = ['trip_id', 'timestamp', 'coords', 'segment', '%', 'bus_stop_number', 'expected_arrival_time', 'delay','accumulated_distance'])
history = np.nan

for row in gps_data2.itertuples(index=False):
   
    point1 = DataPoint(row.timestamp, [row.latitude, row.longitude], row.trip_id, row.bearing)
    gps_history = processed_gps.loc[processed_gps.trip_id == row.trip_id, 'accumulated_distance']
    
    if not gps_history.empty:
        history = gps_history.iloc[-1]  #last accumulated distance for fixed trip_id
    else:
        history = np.nan  

    r = gps_processing_algorithm(point1, bus_stops, kdtree_dict_loaded, segments, history)
    if r is not np.nan:
        processed_gps = pd.concat([processed_gps, pd.DataFrame([r])], ignore_index=True)

In [37]:
processed_gps.iloc[5:10] #sample

,trip_id,timestamp,coords,segment,%,bus_stop_number,expected_arrival_time,delay,accumulated_distance
5,262202411051602_91_262-06,2024-11-05 16:00:02,"[54.38161849975586, 18.60424041748047]",[54.38214092541_18.603386979287_54.38162854692...,0,STOP 1,"[0, 1.9666666666666666]",NaN,[0.08331543238633353]
6,120202411051603_151_120-01,2024-11-05 16:00:02,"[54.32270812988281, 18.6060791015625]",[54.323149127396_18.604743102541_54.3227417640...,0,STOP 1,"[0, 3.966666666666667]",NaN,[0.1416923239018042]
7,6202411051603_21_006-05,2024-11-05 16:00:02,"[54.32278060913086, 18.60675048828125]",[54.323964478265_18.605510822702_54.3226358383...,0,STOP 1,"[0, 2.966666666666667]",NaN,[0.16782987637977909]
8,6202411051601_52_006-09,2024-11-05 16:00:02,"[54.42329025268555, 18.592729568481445]",[54.422538901454_18.591070262415_54.4233306344...,0,STOP 1,"[0, 2.966666666666667]",NaN,[0.14451915387580352]
9,154202411051556_21_154-31,2024-11-05 16:00:02,"[54.35852813720703, 18.64719009399414]",[54.359750515567_18.647868149284_54.3580457892...,0.071044,STOP 2,"[0, -1.0333333333333334]",1.033333,[0.49403047637618724]


In [24]:
processed_gps = processed_gps[~processed_gps['segment'].isnull()]
processed_gps.to_csv('results\\processed_gps.csv', index=False) #saving processed gps readings

visualisation for example trip_id with extra information (green -> segment shape points, red -> gps readings):

In [38]:
import folium
map_gd = folium.Map(location=[54.352, 18.646], zoom_start=13) 

trip = '289202411051536_212_289-01'


counter = 0
for row in segments[segments.shape_id == trips[trips.trip_id == trip].shape_id.iloc[0]].itertuples(index = False):
    counter+=1
    txt = f'{counter},{row.id}, {row.initial_distance}' #segment id, initial distance
    beginning = ast.literal_eval(row.beginning)
    folium.Marker(location = [beginning[0], beginning[1]],
                        popup = txt,
                        icon=folium.Icon(color='green')
                        ).add_to(map_gd)

row = segments[segments.shape_id == trips[trips.trip_id == trip].shape_id.iloc[0]].iloc[-1]
end = ast.literal_eval(row.end)
txt = f'{counter+1}, {row.id}, {row.initial_distance}'

folium.Marker(location = [end[0], end[1]],
                        popup = txt,
                        icon=folium.Icon(color='green')
                        ).add_to(map_gd)



counter2=0
for row in processed_gps[processed_gps.trip_id == trip].itertuples(index = False):
    counter2+=1
    txt = f'{counter2}, {row.segment}, {row.timestamp}, {row.accumulated_distance}' #segment id, timestamp, accumulated distance so far
    
    folium.Marker(location = [row.coords[0], row.coords[1]],
                        popup = txt,
                        icon=folium.Icon(color='red')
                        ).add_to(map_gd)



display(map_gd)

map_gd.save('results\\map_gd.html')